In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score
!pip install xgboost
from xgboost import XGBClassifier
import numpy as np
import os

In [2]:
df = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/FFPE_Discovery_Chowdhury/FFPE_Discovery_normalized.csv",header=0,low_memory=False)
all_prots = df.iloc[:, 1:8801].columns
prots = all_prots.to_list()
print(len(prots))

8800


In [3]:
# 100 tumors here... 
df['Tumor.response'] = df['Tumor.response'].map({'sensitive': 1, 'refractory': 0})

# Separate features and labels
X = df[prots]  # Replace 'label_column' with your label column name
y = df['Tumor.response']  # Replace 'label_column' with your label column name

# Split the data into an 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
rf_pred = rf_clf.predict(X_test)
rf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Evaluate the Random Forest model
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_roc_auc = roc_auc_score(y_test, rf_pred_proba)
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest ROC AUC: {rf_roc_auc}")

# Initialize and train the XGBoost classifier
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

# Make predictions on the test set
xgb_pred = xgb_clf.predict(X_test)
xgb_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Evaluate the XGBoost model
xgb_accuracy = accuracy_score(y_test, xgb_pred)
xgb_roc_auc = roc_auc_score(y_test, xgb_pred_proba)
print(f"XGBoost Accuracy: {xgb_accuracy}")
print(f"XGBoost ROC AUC: {xgb_roc_auc}")

Random Forest Accuracy: 0.75
Random Forest ROC AUC: 0.8333333333333333
XGBoost Accuracy: 0.6
XGBoost ROC AUC: 0.7023809523809523


In [4]:
df = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/HGSOC_multimodal_all_prots_oz_merge.csv")
# df['label'] = df['label'].map({'sensitive': 1, 'refractory': 0})

/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (8807,8811) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# re run but with subset and splits.. 

split_folder_path = '/mnt/ncshare/ozkilim/BRCA/splits/HGSOC_multimodal_100'  # Replace with your actual folder path


rf_test_aucs = []
# Loop over split files and create train, val, test subsets
for filename in os.listdir(split_folder_path):
    if filename.endswith('bool.csv'):  # Assuming the files are in CSV format
        print("split")
        split_df = pd.read_csv(os.path.join(split_folder_path, filename))
        # Extract identifiers
        merged_df = pd.merge(df, split_df, left_on='slide_id', right_on='Unnamed: 0')

        # Create train, validation, and test sets
        train_df = merged_df[merged_df['train']]
        val_df = merged_df[merged_df['val']]
        test_df = merged_df[merged_df['test']]

        X_train = train_df[prots]  # Replace 'label_column' with your label column name
        y_train = train_df['label']  # Replace 'label_column' with your label column name

        X_test = test_df[prots]  # Replace 'label_column' with your label column name
        y_test = test_df['label']  # Replace 'label_column' with your label column name


        # Initialize and train the Random Forest classifier
        rf_clf = RandomForestClassifier()
        rf_clf.fit(X_train, y_train)

        # Make predictions on the test set
        rf_pred = rf_clf.predict(X_test)
        rf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

        # Evaluate the Random Forest model
        rf_accuracy = accuracy_score(y_test, rf_pred)
        rf_roc_auc = roc_auc_score(y_test, rf_pred_proba)
        print(f"Random Forest ROC AUC: {rf_roc_auc}")
        rf_test_aucs.append(rf_roc_auc)
        
    # break
rf_test_aucs = np.array(rf_test_aucs)
print(rf_test_aucs)
print(np.mean(rf_test_aucs))

split


KeyError: "['ADGRE5', 'ADPRS', 'AMY1C', 'ATP5MJ', 'ATP5MK', 'BBLN', 'BPNT2', 'C11orf58', 'C12orf43', 'C12orf75', 'C15orf48', 'C16orf91', 'C17orf49', 'C17orf97', 'C19orf33', 'C1orf116', 'C1orf35', 'CEP43', 'CRACDL', 'CYRIA', 'CYRIB', 'DYNC2I2', 'ELAPOR1', 'EOLA1', 'GET1', 'GET3', 'GFUS', 'GOLM2', 'H1-10', 'H2AC18', 'H2AC20', 'H2AC21', 'H2AC6', 'H2BC10', 'H2BC18', 'H2BC20P', 'H2BC21', 'H2BS1', 'H2BU1', 'H3-4', 'H3C13', 'H4-16', 'H4C7', 'HARS1', 'IRAG1', 'IRAG2', 'MACROH2A2', 'MIX23', 'MYG1', 'PABIR1', 'PABIR2', 'PEDS1', 'POLR1F', 'POLR1G', 'POLR1H', 'RUSF1', 'STEEP1', 'TRAPPC14', 'UTP25'] not in index"